In [1]:
# import os

# os.chdir('src/poc')

In [2]:
import tomllib
import numpy as np
from pprint import pprint
import librosa
import requests
from pydub import AudioSegment
from io import BytesIO
from SpotifyTrack import SpotifyTrack
from poc_test import estimate_tempo
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

auth_manager = None
sp = None

with open('cfg.toml', 'rb') as cfg:
    sptfy_cfg = tomllib.load(cfg)['spotify']
    client_id = sptfy_cfg['client_id']
    client_secret = sptfy_cfg['client_secret']
    auth_manager = SpotifyClientCredentials(client_id=client_id,
                                            client_secret=client_secret)

sp = spotipy.Spotify(auth_manager=auth_manager)

In [ ]:
# The Weeknd's "After Hours" 4yP0hdKOZPNshxUOjY0cZj
#(album_name, album_uri) = ('harvest','spotify:album:7hIjNhXoPqyARBpaKpS3nk')
# (album_name, album_uri) = ('sdp_my_eyes', 'spotify:album:2nhThWvrfDbjXvRfuwlsCE')
#(album_name, album_uri) = ('SOS', 'spotify:album:07w0rG5TETcyihsEIZR3qG')
(album_name, album_uri) = ('Favorite Worst Nightmare (Standard Edition)', 'spotify:album:6rsQnwaoJHxXJRCDBPkBRw')
#(album_name, album_uri) = ('Arcane Season 2 OST', 'spotify:album:23kFyuuRMapEZv6ReQUEit')
# (album_name, album_uri) = ('avril_lavigne', 'spotify:album:3zXjR3y2dUWklKmmp6lEhy')
# (album_name, album_uri) = ('travy_patty', 'spotify:album:18NOKLkZETa4sWwLMIm0UZ')

tracks_raw = sp.album_tracks(album_id=album_uri.split(':')[-1], market='CA')

tracks = [SpotifyTrack(**(tracks_raw['items'][i])) for i in range(len(tracks_raw['items']))]

In [4]:
tracks_raw

{'href': 'https://api.spotify.com/v1/albums/23kFyuuRMapEZv6ReQUEit/tracks?offset=0&limit=50&market=CA',
 'items': [{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6xBZgSMsnKVmaAxzWEwMSD'},
     'href': 'https://api.spotify.com/v1/artists/6xBZgSMsnKVmaAxzWEwMSD',
     'id': '6xBZgSMsnKVmaAxzWEwMSD',
     'name': 'Mike Shinoda',
     'type': 'artist',
     'uri': 'spotify:artist:6xBZgSMsnKVmaAxzWEwMSD'},
    {'external_urls': {'spotify': 'https://open.spotify.com/artist/2j5MUrKK6wSLdUxy2ijqHC'},
     'href': 'https://api.spotify.com/v1/artists/2j5MUrKK6wSLdUxy2ijqHC',
     'id': '2j5MUrKK6wSLdUxy2ijqHC',
     'name': 'Emily Armstrong',
     'type': 'artist',
     'uri': 'spotify:artist:2j5MUrKK6wSLdUxy2ijqHC'}],
   'disc_number': 1,
   'duration_ms': 101533,
   'explicit': False,
   'external_urls': {'spotify': 'https://open.spotify.com/track/4Mb5RufLsk8m6KoXrqHBqV'},
   'href': 'https://api.spotify.com/v1/tracks/4Mb5RufLsk8m6KoXrqHBqV',
   'id': '4Mb5RufLsk8m

In [5]:
tracks

In [6]:
def save_wavs(tracks):
    for track in tracks:
        r = requests.get(track.preview_url)
        audio = AudioSegment.from_mp3(BytesIO(r.content))
        wav_path = f'wavs/{track.name.replace(' ', '').replace('?', '')}.wav'
        audio.export(wav_path, format='wav')

In [7]:
def get_tempo_matt(wav_path):
    signal, fs = librosa.load(wav_path, sr=11025, mono=True)
    signal = signal / np.max(np.abs(signal))
    window_size = 1024
    hop_size = 512
    fs = 11025
    tempo, _, _, _ = estimate_tempo(signal, fs, window_size, hop_size)
    return tempo

In [8]:
acc_tempos = [sp.audio_features(harvest_track.id)[0].get('tempo') for harvest_track in tracks]

In [9]:
# save_wavs(tracks)

In [10]:
matt_tempos = [get_tempo_matt(f'wavs/{track.name.replace(' ', '').replace('?', '')}.wav').item() for track in tracks]

In [11]:
def get_tempo_matt(wav_path):
    signal, fs = librosa.load(wav_path, sr=11025, mono=True)
    signal = signal / np.max(np.abs(signal))

    window_size = 1024
    hop_size = 512
    fs = 11025

    tempo, _, _, _ = estimate_tempo(signal, fs, window_size, hop_size)

    return tempo

In [12]:
pd.DataFrame(zip(matt_tempos, acc_tempos),
             index=(t.name for t in tracks),
             columns=['OurAlgoTempo', 'SpotifyTempo'])

,OurAlgoTempo,SpotifyTempo
Heavy Is The Crown (Original Score) (from the series Arcane League of Legends),215.332031,105.024
I Can't Hear It Now (from the series Arcane League of Legends),1291.992188,119.398
Sucker (from the series Arcane League of Legends),143.554688,148.000
Renegade (We Never Run) (from the series Arcane League of Legends),40.374756,161.077
Hellfire (from the series Arcane League of Legends),75.999540,114.452
To Ashes and Blood (from the series Arcane League of Legends),117.453835,84.899
Paint The Town Blue (from the series Arcane League of Legends),161.499023,82.472
Remember Me (Intro) (from the series Arcane League of Legends),107.666016,110.822
Remember Me (from the series Arcane League of Legends),258.398438,131.221
这样很好 (Isha's Song) (from the series Arcane League of Legends),322.998047,149.917


In [13]:
#compute the average error:
errors = []
for i in range(0, len(acc_tempos)):
    errors.append(abs(matt_tempos[i]-acc_tempos[i])/acc_tempos[i])

#print(errors)

print(f"average error of matt method: {np.mean(errors)*100.0}%")

average error of matt method: 203.53605895513053%
